<a href="https://colab.research.google.com/github/bodorcy/hazifeladatok/blob/main/ml_1_gyakorlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic adathalmaz előfeldolgozása

### Adathalmaz rövid bemutatása

A Titanic adathalmaz az egyik legismertebb nyílt forráskódú dataset, amelyet gyakran használnak adatfeldolgozás, statisztikai elemzés és gépi tanulás gyakorlására. Az adatok az 1912. április 15-én elsüllyedt RMS Titanic utasainak információit tartalmazzák, és azt vizsgálják, hogy kik élték túl a katasztrófát.

### Információk

- Forrás: OpenML, eredetileg a Kaggle és más statisztikai publikációk alapján.
- Egyedek száma: 1309 utas
- Jellemzők száma: 14

### Legfontosabb oszlopok

- pclass =	Utasosztály (1 = első, 2 = második, 3 = harmadosztály)
- sex	= Utas neme (male = férfi, female = nő)
- age	= Utas életkora (néhány érték hiányzik)
- sibsp	= A fedélzeten lévő testvérek és házastársak száma
- parch	= A fedélzeten lévő szülők és gyermekek száma
- fare =	Jegy ára (fontban)
- embarked =	Hol szállt fel az utas (C=Cherbourg, Q=Queenstown, S=Southampton)
- home.dest =	Az utas lakhelye vagy úti célja
- survived =	Célváltozó (0 = nem élte túl, 1 = túlélte)

### Érdekességek a Titanic túlélési adatairól
-  Az 1. osztályú utasok túlélési aránya sokkal magasabb volt, mint a 3. osztályú utasoké.
-  A nők túlélési esélye jelentősen nagyobb volt, mint a férfiaké.
-  Az "Women and children first" szabály miatt a fiatalabb utasok nagyobb valószínűséggel élték túl a katasztrófát.

### 1. Feladat – Adatok betöltése és előzetes elemzése


- Töltsd be a Titanic adathalmazt a Scikit-learn fetch_openml függvényével.
- Ellenőrizd az adathalmaz méretét!
- Nézd meg az első 5 sort és az oszlopneveket (columns).

Elvárt kimenet (példa, a pontos értékek változhatnak):

```
Az adathalmaz mérete: (1309, 14)
Oszlopnevek: ['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest']
```

<br>

Segítség a beolvasáshoz:

```
# openml-hez szükséges csomag
from sklearn.datasets import fetch_openml

# Adatbázis betöltése az OpenML-ből
titanic = fetch_openml(name="titanic", version=1, as_frame=True)

# Jellemzővektorok Pandas DataFrame formátumban
df = titanic.data

# A célváltozót (y) is hozzárendeljük
df["survived"] = titanic.target
```

### 2. Feladat – Hiányzó értékek kezelése


- Határozd meg, mely jellemzők tartalmaznak hiányzó adatokat.
- Pótold a numerikus jellemzők hiányzó értékeit a jellemző átlagával.
    ```
    df.select_dtypes(include=["number"])
    ```
- Pótold a kategórikus oszlopokban lévő hiányzó értékeket a leggyakoribb értékkel (lásd [módusz](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html)).
    ```
    df.select_dtypes(exclude=["number"])
    ```


In [78]:
# openml-hez szükséges csomag
from sklearn.datasets import fetch_openml

# Adatbázis betöltése az OpenML-ből
titanic = fetch_openml(name="titanic", version=1, as_frame=True)

# Jellemzővektorok Pandas DataFrame formátumban
df = titanic.data

# A célváltozót (y) is hozzárendeljük
df["survived"] = titanic.target

num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = list(set(df.columns.tolist()) - set(num_cols))

values = [df.loc[:,v].mean() if v in num_cols else df.loc[:,v].mode()[0] for v in df.columns.tolist()]
replace = {col:value for col,value in zip(df.columns.tolist(), values)}

df.fillna(value=replace).head()

#numeric = df.select_dtypes(include=["number"])


,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,survived
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,160.809917,"St Louis, MO",1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,160.809917,"Montreal, PQ / Chesterville, ON",1
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,13,160.809917,"Montreal, PQ / Chesterville, ON",0
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,13,135.000000,"Montreal, PQ / Chesterville, ON",0
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,13,160.809917,"Montreal, PQ / Chesterville, ON",0


### 3. Feladat – Kategóriák numerikussá alakítása
Feladat:

- Alakítsd át a "sex" oszlopot Label Encoding segítségével (male=0, female=1).
- Használj One-Hot Encodingot az "embarked" oszlopra (S, C, Q).
    ```
    pd.get_dummies() --> data, columns, dtype mezőket töltsd fel
    ```

In [79]:
import pandas as pd

df['sex'] = df['sex'].apply(lambda x: 1 if x in ('female', 1) else 0)

pd.get_dummies(df,columns=['embarked'])

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,boat,body,home.dest,survived,embarked_C,embarked_Q,embarked_S
0,1,"Allen, Miss. Elisabeth Walton",1,29.0000,0,0,24160,211.3375,B5,2,NaN,"St Louis, MO",1,False,False,True
1,1,"Allison, Master. Hudson Trevor",0,0.9167,1,2,113781,151.5500,C22 C26,11,NaN,"Montreal, PQ / Chesterville, ON",1,False,False,True
2,1,"Allison, Miss. Helen Loraine",1,2.0000,1,2,113781,151.5500,C22 C26,NaN,NaN,"Montreal, PQ / Chesterville, ON",0,False,False,True
3,1,"Allison, Mr. Hudson Joshua Creighton",0,30.0000,1,2,113781,151.5500,C22 C26,NaN,135.0,"Montreal, PQ / Chesterville, ON",0,False,False,True
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",1,25.0000,1,2,113781,151.5500,C22 C26,NaN,NaN,"Montreal, PQ / Chesterville, ON",0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Zabour, Miss. Hileni",1,14.5000,1,0,2665,14.4542,NaN,NaN,328.0,NaN,0,True,False,False
1305,3,"Zabour, Miss. Thamine",1,NaN,1,0,2665,14.4542,NaN,NaN,NaN,NaN,0,True,False,False
1306,3,"Zakarian, Mr. Mapriededer",0,26.5000,0,0,2656,7.2250,NaN,NaN,304.0,NaN,0,True,False,False
1307,3,"Zakarian, Mr. Ortin",0,27.0000,0,0,2670,7.2250,NaN,NaN,NaN,NaN,0,True,False,False


### 4. Feladat – Csoportosítás
Feladat:

- Határozd meg, hogy a túlélési arány különbözik-e az utasosztályok (pclass) szerint.
- Nézd meg a teljes DataFrame-re a túlélési arányt.
    ```
    df["survived"].value_counts()
    ```
- Csoportosítsd az adatokat osztály (pclass) szerint, és számítsd ki a túlélési arányt csoportonként.
    ```
    df_1 = df.loc[df["pclass"] == 1]
    ```

### 5. Feladat – Adatok szűrése és rendezése
Feladat:

- Szűrd ki a 30 év alatti nőket, akik 1. osztályon utaztak, és túlélték a katasztrófát.
- Rendezd az eredményt kor szerint csökkenő sorrendbe.